In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import pysentiment2 as ps
import nltk
import pysentiment2 as ps
from nltk.stem.porter import *
import os
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

In [2]:
#Load the Returns file
df = pd.read_csv('../DataRetrieval/url_with_returns.csv')[['reportDate','filingDate', 'cik', 'ExcessReturnsEqualWeightedSnP', 'ExcessReturnsValueWeightedSnP']]
df['cik'] = df['cik'].astype(np.int).astype(str).str.pad(10, fillchar='0')
# Filtering out returns on or after 2016-01-01
df = df[df.reportDate >= '2016-01-01']
print(df.shape)

(11383, 5)


C:\Users\JuliJaramillo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\JuliJaramillo\AppData\Local\Temp\ipykernel_8464\151399494.py:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df['cik'] = df['cik'].astype(np.int).astype(str).str.pad(10, fillchar='0')


In [18]:
df

,reportDate,filingDate,cik,ExcessReturnsEqualWeightedSnP,ExcessReturnsValueWeightedSnP,propLM,propHar,weightedLM,weightedHar
20,2016-03-31,2016-05-09,0000814453,0.015216,0.012411,0.027910,0.074000,0.029333,0.026957
21,2016-06-30,2016-08-09,0000814453,0.010468,0.012448,0.026423,0.073618,0.022645,0.021967
22,2016-09-30,2016-11-08,0000814453,-0.077190,-0.069763,0.028069,0.076594,0.024874,0.025146
23,2016-12-31,2017-03-01,0000814453,-0.000803,-0.003919,0.028890,0.075044,0.024955,0.027954
24,2017-03-31,2017-05-10,0000814453,0.001553,-0.002548,0.025452,0.075698,0.030192,0.030750
...,...,...,...,...,...,...,...,...,...
20726,2021-09-30,2021-11-04,0001682852,-0.174756,-0.166301,0.020214,0.072817,0.006567,0.032208
20727,2021-09-30,2021-11-08,0000842023,0.004587,0.011895,0.014241,0.064210,0.006367,0.008581
20728,2021-09-30,2021-11-08,0000891103,-0.035377,-0.028070,0.035195,0.072138,0.037698,0.033316
20729,2021-09-30,2021-11-03,0001725057,-0.039414,-0.041318,0.023269,0.081838,0.017123,0.020058


In [3]:
#Define the dictionaries

LMDictionary = pd.read_excel('LM_Neg.xlsx', sheet_name = 'ND_FinTerms_Negative_v2', header = None).values.reshape(-1)
LMDictionary = set( [w.lower().strip() for w in LMDictionary ] )

HarvardDictionary = list(open('Harvard_Neg_Inf.txt', 'r'))
HarvardDictionary = set ( [w.strip().lower() for w in HarvardDictionary ] )


#Define the stopwords dictionary
stop_words = list(set(stopwords.words('english')))

C:\Users\JuliJaramillo\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [4]:
print('Length of Harvard,LM and stopwords is', len(LMDictionary ), len(HarvardDictionary), len(stop_words))

Length of Harvard,LM and stopwords is 2337 4187 179


In [5]:
def lemmatize_words(words):
    lemmatized_words = [WordNetLemmatizer().lemmatize(word, 'v') for word in words]
    return lemmatized_words

In [6]:
# Lemmatize Dictionaries and stop words
stop_words = set(lemmatize_words(stopwords.words('english')))
HarvardDictionary = set(lemmatize_words(HarvardDictionary))
LMDictionary = set(lemmatize_words(LMDictionary))

In [7]:
print('Length of Harvard, LM and stopwords is', len(LMDictionary ), len(HarvardDictionary), len(stop_words))

Length of Harvard,LM and stopwords is 1495 2388 166


In [8]:
#listFileNames = os.listdir('../Forms/')
listFileNames = os.listdir('../DataRetrieval/Forms/')

# Calculates the total number of documents under consideration
N = 0

# Calculate the number of documents in which a word has appeared 
globalDict = {}

This loop computes the value of N and compute a dictionary that counts the number of documents in which a word has appeared.

In [9]:
for row, fileName in enumerate(listFileNames):
    
    cik = fileName.split('_')[0]
    filingDate = fileName.split('_')[1].split('.')[0]
    #path = '../Forms/{}_{}.txt'.format(cik, filingDate)

    path = '../DataRetrieval/Forms/{}_{}.txt'.format(cik, filingDate)
    
    f = open(path, "r")
    text = f.read()
    
    tokensOwn = nltk.word_tokenize(text)
    tokensOwn= list(lemmatize_words(tokensOwn))

    # Filtering out the documents where the numbers of words are less than 2000
    if len(tokensOwn) >= 2000:       
        N+=1
        filteredText = [w for w in tokensOwn if not w in stop_words]
        
        # Updated the global_dict to include count of words observed in the current document
        #Number of documents that contains the token "word"
        #Df_i
        for word in list(set(filteredText)):
            if not word in globalDict:
                globalDict[word] = 1
            else:
                globalDict[word] += 1
        
    f.close()
    
    if (row % 1000 == 0) & (row != 0): 
        print(row)


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000


In [10]:
print('Total number of documents under consideration:', N)

Total number of documents under consideration: 11360


In [11]:
print('Total number of distinct words across all documents:', len(globalDict.keys()))

Total number of distinct words across all documents: 245178


In [12]:
def wordCountDictionary(documentTokens):
    '''
    Counting occurrences of a given word in a document. EVERY WORD, not just the negative words. (TF_ij)
    documentTokens: (list) tokens of the document
    '''
    
    documentWordCountDict = {}
    for word in list(documentTokens):
        if not word in documentWordCountDict:
            documentWordCountDict[word] = 1
        else:
            documentWordCountDict[word] += 1
            
    return documentWordCountDict     


In [13]:
def countNegativeWords(documentTokens, setNegativeWords):
    '''
    Counts negative words specified in setNegativeWords given the documentTokens
    '''
    countNegativeWords = 0
    for word in documentTokens:
        if word in setNegativeWords:
            countNegativeWords+=1
    return countNegativeWords

In [14]:
def weightedNegativeWords(documentTokens, setNegativeWords, wordWeights):
    '''
    Counts weighted (wordWeights) proportion of negative words specified in setNegativeWords given the documentTokens
    '''
    
    weightedNegativeWords = 0.0
    weightedWords = 0.0
    for word in documentTokens:
        weightedWords += wordWeights[word]
        if word in setNegativeWords:
            weightedNegativeWords += wordWeights[word]
            
    return weightedNegativeWords/weightedWords
    

This loop iterates over all documents and computes the proportion of negative words both unweighted and weighted using the Harvard and LM Dictionaries. 

In [15]:
for row, fileName in enumerate(listFileNames):
    
    cik = fileName.split('_')[0]
    filingDate = fileName.split('_')[1].split('.')[0]
    #path = '../Forms/{}_{}.txt'.format(cik, filingDate)

    path = '../DataRetrieval/Forms/{}_{}.txt'.format(cik, filingDate)
    
    f = open(path, "r")
    text = f.read()
    
    tokensOwn = nltk.word_tokenize(text)
    tokensOwn= list(lemmatize_words(tokensOwn))

    # Filtering out the documents where the numbers of words are less than 2000
    if len(tokensOwn) >= 2000:       
        filteredText = [w for w in tokensOwn if not w in stop_words]
        
        numeratorLM = countNegativeWords(filteredText, LMDictionary)
        propLM = numeratorLM/len(filteredText)
        
        numeratorHar = countNegativeWords(filteredText, HarvardDictionary)
        propHar = numeratorHar/len(filteredText)
        
        df.loc[(df['cik']== cik) & (df['filingDate'] == filingDate), 'propLM'] = propLM 
        df.loc[(df['cik']== cik) & (df['filingDate'] == filingDate), 'propHar'] = propHar
        
        
        wCountDictionary = wordCountDictionary(filteredText)
        a = np.mean(list(wCountDictionary.values()))
        
        wordWeights = {}
        
        for word, word_count in wCountDictionary.items():
            wordWeights[word] = (1 + np.log(wCountDictionary[word]))*(np.log(N/globalDict[word]))/(1 + np.log(a))
        
        
        weightedLM = weightedNegativeWords(filteredText, LMDictionary, wordWeights)
        weightedHar = weightedNegativeWords(filteredText, HarvardDictionary, wordWeights)
        
        df.loc[(df['cik']== cik) & (df['filingDate'] == filingDate), 'weightedLM'] = weightedLM 
        df.loc[(df['cik']== cik) & (df['filingDate'] == filingDate), 'weightedHar'] = weightedHar 
                
    f.close()
    
    if (row % 1000 == 0) & (row != 0): 
        print(row)


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000


In [16]:
final = df[df['propLM'].notna()]

In [17]:
final.to_csv('proportions.csv')